In [131]:
import os
import pandas

In [132]:
FILENAME = 'json/club2018/108ff560de.json'

TEMP_NAME = os.path.basename(FILENAME)

ROSTER_PAGE = 'http://clubworldchampionships.2018.women.fivb.com/en/teams/alt%20altay%20volleyball%20club%20kaz/team_roster'

In [133]:
data = pandas.read_json(FILENAME)
df = pandas.DataFrame(data)


In [134]:
import requests
from bs4 import BeautifulSoup

response = requests.get(ROSTER_PAGE)
print(response.status_code)
soup = BeautifulSoup(response.text, 'html.parser')

200


In [135]:
from pandas import Series

links = soup.find_all('a')

def links_with_player(link):
    try:
        href = link['href']
    except:
        return False
    else:
        if '/players/' in href:
            return True
        else:
            return False

filtered_links = filter(links_with_player, links)
hrefs = list(map(lambda x: (x.text, x['href']), filtered_links))

In [136]:
from w3lib.url import canonicalize_url, urljoin

container = []

for name in df['name']:
    for href in hrefs:
        if name in href:
            name, path = href
            url = canonicalize_url(urljoin('http://clubworldchampionships.2018.women.fivb.com/', path))
            container.append(url)

In [137]:
df['player_page'] = Series(container, dtype=str)
df.head()

,name,date_of_birth,height,weight,spike,block,player_page,curr_age,competition_name,competition_year,team,player_image,nationality,position,short_position
0,Natalya Mammadova,1984-01-02,196,78,319,302,http://clubworldchampionships.2018.women.fivb....,37,Club Word Championships,2018,NaN,NaN,NaN,NaN,NaN
1,Sana Anarkulova C,1989-01-21,188,77,300,280,http://clubworldchampionships.2018.women.fivb....,32,Club Word Championships,2018,NaN,NaN,NaN,NaN,NaN
2,Zarina Sitkazinova,1993-01-20,182,70,295,280,http://clubworldchampionships.2018.women.fivb....,28,Club Word Championships,2018,NaN,NaN,NaN,NaN,NaN
3,Aliya Sadykova,1988-01-01,174,63,293,281,http://clubworldchampionships.2018.women.fivb....,33,Club Word Championships,2018,NaN,NaN,NaN,NaN,NaN
4,Kristina Belova,1998-01-29,182,72,244,285,http://clubworldchampionships.2018.women.fivb....,23,Club Word Championships,2018,NaN,NaN,NaN,NaN,NaN


In [138]:
df.to_json(FILENAME, orient='records')

In [139]:
from w3lib.url import add_or_replace_parameter
from zineb.utils._html import deep_clean

positions = []
images = []
for url in df['player_page']:
    try:
        player_page = requests.get(url)
    except:
        images.append(None)
        positions.append(None)
    else:
        soup = BeautifulSoup(player_page.text, 'html.parser')
        if soup is not None:
            image_div = soup.find('div', attrs={'class': 'person'})
            src = add_or_replace_parameter(canonicalize_url(image_div.find('img')['src']), 'height', 2000)
            images.append(src)

            line_list = soup.find('ul', attrs={'class': 'line-list'})
            try:
                position = line_list.find_all('li')[0].find_next('strong').text
            except:
                position = None
            positions.append(deep_clean(position))
        else:
            images.append(None)
            positions.append(None)
    

df['position'] = Series(positions, dtype=str)
df['player_image'] = Series(images, dtype=str)


In [141]:
df.to_json('108ff560de.json', orient='records')

['108ff560de', 'json']